In [60]:
import pandas as pd
import numpy as np

In [22]:
data_demo = pd.read_csv("./data/data/bitcoin_markets_daily_discussion_january_v1.csv")

In [23]:
data_demo.iloc[0]

comment_id                                                         dtk204c
daily_discussion_date                          Wednesday, January 31, 2018
created                                                        1.51746e+09
body                     #New post: [\[Daily Discussion\] Thursday, Feb...
parent_id                                                           7u762z
vader_scores             {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
upvotes                                                                  1
downvotes                                                                0
author_opinion                                                        None
Name: 0, dtype: object

In [8]:
eval(data_demo['vader_scores'].iloc[9])

{'compound': 0.7894, 'neg': 0.129, 'neu': 0.556, 'pos': 0.314}

In [27]:
# turn the vader dict columns into four seperate cols.
def vader_score(dataframe):
    text = pd.DataFrame({'vader_negative':[0]*dataframe.shape[0],'vader_neutral':[0]*dataframe.shape[0],
                         'vader_positive':[0] * dataframe.shape[0], 'vader_compound':[0]*dataframe.shape[0]})
    for i in range(dataframe.shape[0]):
        text['vader_negative'].iloc[i] = eval(data_demo['vader_scores'].iloc[i])['neg']
        text['vader_positive'].iloc[i] =  eval(data_demo['vader_scores'].iloc[i])['pos']
        text['vader_neutral'].iloc[i] =  eval(data_demo['vader_scores'].iloc[i])['neu']
        text['vader_compound'].iloc[i] =  eval(data_demo['vader_scores'].iloc[i])['compound']
    return text

In [28]:
data_demo = pd.merge(data_demo, vader_score(data_demo), left_index=True, right_index=True)
data_demo.head(5)

D:\Programming\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,comment_id,daily_discussion_date,created,body,parent_id,vader_scores,upvotes,downvotes,author_opinion,vader_compound,vader_negative,vader_neutral,vader_positive
0,dtk204c,"Wednesday, January 31, 2018",1.517459e+09,"#New post: [\[Daily Discussion\] Thursday, Feb...",7u762z,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",1,0,None,0.0000,0.000,1.000,0.000
1,dticvel,"Wednesday, January 31, 2018",1.517384e+09,With over 600 million registered users and 200...,7u762z,"{'neg': 0.0, 'neu': 0.906, 'pos': 0.094, 'comp...",73,0,None,0.4019,0.000,0.906,0.094
2,dtj92lt,"Wednesday, January 31, 2018",1.517428e+09,I tried to post a long writeup with my thought...,7u762z,"{'neg': 0.0, 'neu': 0.909, 'pos': 0.091, 'comp...",37,0,None,0.4019,0.000,0.909,0.091
3,dtiiuq4,"Wednesday, January 31, 2018",1.517399e+09,Just to straighten out some facts:\t\t- canary...,7u762z,"{'neg': 0.085, 'neu': 0.855, 'pos': 0.06, 'com...",95,0,None,-0.9165,0.085,0.855,0.060
4,dti9rd0,"Wednesday, January 31, 2018",1.517378e+09,"Ahh, the roller coaster of bitcoin.\t\tFuck it...",7u762z,"{'neg': 0.296, 'neu': 0.704, 'pos': 0.0, 'comp...",26,0,None,-0.5848,0.296,0.704,0.000


In [29]:
from datetime import datetime
def turn_datetime(dataframe):
    t = []
    for i in range(dataframe.shape[0]):
        Month = dataframe['daily_discussion_date'].iloc[i].split(',')[1].split(' ')[1][0:3]
        Day = dataframe['daily_discussion_date'].iloc[i].split(',')[1].split(' ')[2]
        year = dataframe['daily_discussion_date'].iloc[i].split(',')[2].split(' ')[-1]
        datetime_object = datetime.strptime(Month + ' ' + Day + ' ' + year, '%b %d %Y')
        t.append(datetime_object)
    dataframe['daily_discussion_date'] = pd.DataFrame(t)

turn_datetime(data_demo)

In [24]:
'''
def average_Score(dataframe, interval):
    
    if interval < 1 or interval > 24:
        print("Wrong interval")
    t = pd.DataFrame({'positive':[0] * (dataframe.shape[0]),
                      'negative':[0] * (dataframe.shape[0]),
                      'neutral':[0] * (dataframe.shape[0]),
                      'compound':[0] * (dataframe.shape[0])})
    for i in range(interval, dataframe.shape[0]):
        avg_pos = dataframe['positive'].iloc[i-interval: i].mean()
        avg_neg = dataframe['negative'].iloc[i-interval: i].mean()
        avg_neu = dataframe['neutral'].iloc[i-interval: i].mean()
        avg_com = dataframe['compound'].iloc[i-interval: i].mean()
        #按照字母顺序，依次是 compound, negative, neutral, positive
        t.iloc[i] = avg_com, avg_neg, avg_neu, avg_pos
    return t
'''

This is the function that allows you to get daily average of four sentiment scores.

In [36]:
# will do a demo of daily here. Will modify later to see whether we use hourly or 6-hour interval.
def sentiment_normal_avg_byday(df, lexicon_name):
    # lexicon name can only be "vader", "socialsent".
    sent_catogory = ["compound", "neutral", "negative", "positive"]
    col_names = [lexicon_name+"_"+i for i in sent_catogory]
    groupby_dict = {}
    for key in col_names:
        groupby_dict[key] = "mean"
    return data_demo.groupby("daily_discussion_date").agg(groupby_dict)

In [117]:
daily_feature = sentiment_normal_avg_byday(data_demo, "vader")

In [43]:
daily_feature.head()

,vader_neutral,vader_negative,vader_positive,vader_compound
daily_discussion_date,,,,
2018-01-01,0.805734,0.071214,0.123046,0.126620
2018-01-02,0.819752,0.065844,0.113365,0.131937
2018-01-03,0.813199,0.075551,0.111242,0.123640
2018-01-04,0.813771,0.073283,0.112134,0.083307
2018-01-05,0.804517,0.069134,0.126343,0.136669


Then we get average of sentiment scores based on num of upvotes. To have a quick check:

In [59]:
data_demo["upvotes"].describe()

count    46712.000000
mean         3.027102
std          5.191137
min        -45.000000
25%          1.000000
50%          2.000000
75%          4.000000
max        148.000000
Name: upvotes, dtype: float64

It seems we have both pos, 0 and neg. Just use the number as weight!

In [67]:
def sentiment_votes_avg_byday(df, lexicon_name):
    weighted_mean = lambda x: np.average(x, weights=df.loc[x.index, "upvotes"])
    # lexicon name can only be "vader", "socialsent".
    sent_catogory = ["compound", "neutral", "negative", "positive"]
    col_names = [lexicon_name+"_"+i for i in sent_catogory]
    renames = [lexicon_name+"_"+"votes"+"_"+i for i in sent_catogory]
    groupby_dict = {}
    for key in col_names:
        groupby_dict[key] = weighted_mean
    df_result = data_demo.groupby("daily_discussion_date").agg(groupby_dict)
    df_result.columns = renames
    return df_result

In [118]:
daily_feature = pd.concat([daily_feature, sentiment_votes_avg_byday(data_demo, "vader")], axis=1)
daily_feature

,vader_neutral,vader_negative,vader_positive,vader_compound,vader_votes_compound,vader_votes_neutral,vader_votes_negative,vader_votes_positive
daily_discussion_date,,,,,,,,
2018-01-01,0.805734,0.071214,0.123046,0.126620,0.812685,0.057980,0.129303,0.198863
2018-01-02,0.819752,0.065844,0.113365,0.131937,0.827305,0.060349,0.111737,0.184096
2018-01-03,0.813199,0.075551,0.111242,0.123640,0.804258,0.083496,0.112270,0.150481
2018-01-04,0.813771,0.073283,0.112134,0.083307,0.810787,0.080932,0.107818,0.098907
2018-01-05,0.804517,0.069134,0.126343,0.136669,0.808516,0.068140,0.123352,0.179115
2018-01-06,0.798341,0.073268,0.127214,0.145679,0.798569,0.060172,0.138766,0.211830
2018-01-07,0.792146,0.084094,0.123753,0.097713,0.797649,0.081457,0.120864,0.129741
2018-01-08,0.796422,0.079094,0.123797,0.095672,0.799486,0.077610,0.122449,0.104333
2018-01-09,0.798485,0.079781,0.120771,0.107711,0.801833,0.079140,0.118187,0.158298


Great! Now let's get number of children for each sample (row).

In [73]:
# this might run long
%%time
data_demo["num_child"] = data_demo["comment_id"].apply(lambda x: data_demo[data_demo["parent_id"]==x].shape[0])

In [77]:
data_demo["num_child"].describe()

count    46712.000000
mean         0.832484
std          1.171531
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max         42.000000
Name: num_child, dtype: float64

#### Note that this is the next-level comment under each comment, not the total comment (total number of notes) under this comment. Maybe under methods (tree-like?) can be used later. But at this time we just use this as a weight for sentiment scores and feature.

In [78]:
def sentiment_child_avg_byday(df, lexicon_name):
    weighted_mean = lambda x: np.average(x, weights=df.loc[x.index, "num_child"])
    # lexicon name can only be "vader", "socialsent".
    sent_catogory = ["compound", "neutral", "negative", "positive"]
    col_names = [lexicon_name+"_"+i for i in sent_catogory]
    renames = [lexicon_name+"_"+"child"+"_"+i for i in sent_catogory]
    groupby_dict = {}
    for key in col_names:
        groupby_dict[key] = weighted_mean
    df_result = data_demo.groupby("daily_discussion_date").agg(groupby_dict)
    df_result.columns = renames
    return df_result

In [119]:
daily_feature = pd.concat([daily_feature, sentiment_child_avg_byday(data_demo, "vader")], axis=1)
daily_feature.head()

,vader_neutral,vader_negative,vader_positive,vader_compound,vader_votes_compound,vader_votes_neutral,vader_votes_negative,vader_votes_positive,vader_child_compound,vader_child_neutral,vader_child_negative,vader_child_positive
daily_discussion_date,,,,,,,,,,,,
2018-01-01,0.805734,0.071214,0.123046,0.126620,0.812685,0.057980,0.129303,0.198863,0.831541,0.074968,0.093485,0.105663
2018-01-02,0.819752,0.065844,0.113365,0.131937,0.827305,0.060349,0.111737,0.184096,0.832181,0.063824,0.103987,0.159920
2018-01-03,0.813199,0.075551,0.111242,0.123640,0.804258,0.083496,0.112270,0.150481,0.832627,0.073780,0.093607,0.138089
2018-01-04,0.813771,0.073283,0.112134,0.083307,0.810787,0.080932,0.107818,0.098907,0.834811,0.073298,0.091908,0.068976
2018-01-05,0.804517,0.069134,0.126343,0.136669,0.808516,0.068140,0.123352,0.179115,0.819114,0.070553,0.110307,0.167191


#### Then let's get statistical features:
1. number of comments in a day
2. number of child comments fo a comment in a day
3. avg length of text
4. proportion of types of user (bullish, ...) in a day

In [109]:
# Bullish, Bearish, Long-term Holder, Bitcoin Skeptic, None
def get_author_opinion(dataframe):
    opinion_types = ['Bullish','Bearish','Long-term Holder','Bitcoin Skeptic','None']
    for type in opinion_types:
        dataframe[type] = 0
    for i in range(dataframe.shape[0]):
        dataframe.set_value(i, dataframe.iloc[i]["author_opinion"], 1)
    return dataframe

data_demo = get_author_opinion(data_demo)

In [114]:
def opinion_avg_byday(df):
    opinion_types = ['Bullish','Bearish','Long-term Holder','Bitcoin Skeptic','None']
    groupby_dict = {}
    for type in opinion_types:
        groupby_dict[type] = "mean"
    return data_demo.groupby("daily_discussion_date").agg(groupby_dict)

In [120]:
df_num_comments = data_demo.groupby("daily_discussion_date").size().to_frame()
df_num_comments.columns = ["num_comments"]

df_num_children = data_demo.groupby("daily_discussion_date").agg({"num_child":"mean"})

data_demo["num_word"] = data_demo["body"].apply(lambda x:len(x))
df_num_word = data_demo.groupby("daily_discussion_date").agg({"num_word":"mean"})

daily_feature = pd.concat([daily_feature, opinion_avg_byday(data_demo), 
                           df_num_comments, df_num_children, df_num_word], axis=1)

In [124]:
daily_feature.shape

(31, 20)